In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

Подготовка датасета

In [2]:
# Определим преобразования для данных
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Загрузим обучающий и тестовый наборы данных
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Разделим обучающий набор на обучающую и валидационную выборки
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Определим загрузчики данных
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 52231711.67it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1806350.84it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 12623326.50it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 6654044.28it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Создание класса простой полносвязной нейронной сети

In [3]:
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Инициализация модели, критерия и оптимизатора
model1 = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)


In [4]:
def train(model, optimizer, criterion, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss/len(val_loader)}")


In [5]:
# Обучение полносвязанной нейронной сети
train(model1, optimizer1, criterion, train_loader, val_loader, epochs=10)

Epoch 1/10, Training Loss: 0.22698038081079722, Validation Loss: 0.1256291949081215
Epoch 2/10, Training Loss: 0.0939226277495424, Validation Loss: 0.1145543840396753
Epoch 3/10, Training Loss: 0.06662066135027757, Validation Loss: 0.11324446808368444
Epoch 4/10, Training Loss: 0.04855739006520404, Validation Loss: 0.09792500184576443
Epoch 5/10, Training Loss: 0.03802499145960125, Validation Loss: 0.08467931223369421
Epoch 6/10, Training Loss: 0.03131451450916938, Validation Loss: 0.12517419279964165
Epoch 7/10, Training Loss: 0.02882078992992562, Validation Loss: 0.09435964111013319
Epoch 8/10, Training Loss: 0.024709180767609116, Validation Loss: 0.10892689883807412
Epoch 9/10, Training Loss: 0.022180620793020352, Validation Loss: 0.10152326824901776
Epoch 10/10, Training Loss: 0.0225430518453965, Validation Loss: 0.10716780891070425


In [6]:
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total}%")

# Тестирование полносвязанной нейронной сети
test(model1, test_loader)

Accuracy: 97.69%


В этой работе мы обучили нейронную сеть распознавать рукописные буквы, используя датасет MNIST. Датасет состоит из изображений цифр от 0 до 9 размером 28x28 пикселей. Мы использовали библиотеку torchvision для загрузки и предварительной обработки данных.

**Преобразования и загрузка данных**

Мы применили следующие преобразования к изображениям:

1.   Преобразование изображения в тензор с помощью ToTensor
2.   Нормализация данных с помощью Normalize

Затем мы загрузили обучающий и тестовый наборы данных с использованием функции torchvision.datasets.MNIST. Обучающий набор данных был разделен на две части: 80% данных использовались для обучения, а оставшиеся 20% — для валидации. Это было сделано с помощью функции random_split.
Для эффективного обучения и тестирования мы использовали DataLoader с размером батча 64.

**Архитектура модели**

Мы создали простую полносвязанную нейронную сеть SimpleNN, состоящую из трех полносвязанных слоев:

Первый слой: вход 28*28 нейронов, выход 512 нейронов
Второй слой: вход 512 нейронов, выход 256 нейронов
Третий слой: вход 256 нейронов, выход 10 нейронов (по числу классов)
Для активации использовалась функция ReLU.

Мы использовали функцию потерь CrossEntropyLoss и оптимизатор Adam с начальной скоростью обучения 0.001.

**Процесс обучения**
Обучение модели проводилось в течение 10 эпох. В каждой эпохе:

1. Модель обучалась на обучающем наборе данных, и накапливалась средняя величина функции потерь
2. Модель оценивалась на валидационном наборе данных, и также вычислялась средняя величина функции потерь

В процессе обучения наблюдалось следующее:

* Средняя величина функции потерь на обучающем наборе данных постепенно уменьшалась.
* Величина функции потерь на валидационном наборе данных не показывала значительного роста, что указывает на отсутствие переобучения.

**Тестирование**

Для того, чтобы провести тестирование мы:

1. Переключаем модель в режим оценки методом model.eval()
2. Отключаем градиентное вычисление методом torch.no_grad(), так как нам не нужны градиенты для оценки модели
3. В цикле мы перебираем все батчи изображений и меток из test_loader
4. Получаем предсказания модели
5. Сравниваем предсказанные модели с истинными значениями. Метрика Accuracy считает отношение верно угаданных значений к общему количеству.

Точность модели составила **97.69%**

Это довольно хороший показатель, поэтому мы можем сказать, что наша архитектура простой полносвязанной нейронной сети оказалась крайне эффективна для задачи распознавания рукописных цифр.